In [1]:
import morfeusz2
import pandas as pd

In [2]:
m = morfeusz2.Morfeusz()

In [3]:
articles = pd.read_csv('labeling/fresh_data.csv')\
    [['id', 'title', 'highlight', 'content', 'media_desc']]\

In [4]:
articles.head(1)

,id,title,highlight,content,media_desc
0,24247530,czas wyczekiwany final dekomunizacji ulic piat...,naczelny sad administracyjny rozny finalizuje ...,piatkowej wokandzie nsa figuruje nazw stoleczn...,marzec zakladanie nowych tablic dawnej al armi...


In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35233 entries, 0 to 35232
Data columns (total 5 columns):
id            35233 non-null int64
title         35231 non-null object
highlight     35231 non-null object
content       35225 non-null object
media_desc    31554 non-null object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [6]:
from unidecode import unidecode

In [7]:
def lemmatize_column(column):
    lemmatized_column = [] 
    for record in column.values:
        lemmatized_row = []
        try:
            for word in record.split(' '):
                lemmatized_row.append(unidecode(m.analyse(word)[0][2][1].split(':')[0]).lower())
            lemmatized_column.append(" ".join(lemmatized_row))
        except:
            lemmatized_column.append(None)
        
    return pd.Series(lemmatized_column, name = column.name)

In [8]:
lem_df_list = []
for column in [u'content', u'highlight', u'media_desc', u'title']:
    lem_df_list.append(lemmatize_column(articles[column]))
lem_df_list.append(articles['id'])

In [9]:
lem_df = pd.concat(lem_df_list, axis = 1)

In [10]:
lem_df.head(5)

,content,highlight,media_desc,title,id
0,piatkowej wokanda nsa figurowac nazwa stoleczn...,naczelny sad administracyjny rozny finalizowac...,marzec zakladanie nowy tablica dawny al armia ...,czas wyczekiwac final dekomunizacja ulica piat...,24247530
1,mezczyzni dzialali zorganizowac grupa przestep...,rzeszowski delegatura centralny biuro antykoru...,cba,cba zatrzymalo pieciu podkarpacki przedsiebior...,24593465
2,kazde morderstwo tragedia pawela adamowicz wie...,stanac pogodzic mowa nienawisci rozprzestrzeni...,prezydent miasto pawela adamowicz zyje,swieca pawla wyprzec ciemnosc,24376781
3,letni jacek kuzminski mieszkac wielki brytania...,spodziewalem taki odzew nadziac zly mowi jacek...,jacek kuzminski lewy pomagajacy lukasz grodzki,pobic oksford polak stracil prace wyladowal ul...,22852664
4,wedlug dany opublikowac poniedzialek irlandzki...,niepewnosc zwiazana wyjsciem wielki brytania u...,brexit rosnie liczba brytyjczykow ubiegajacych...,brexit rosnie liczba brytyjczykow ubiegajacych...,24320164


In [11]:
from collections import Counter

In [12]:
lem_df = lem_df.fillna('')

In [13]:
bag_of_records = lem_df['title'] + ' ' + lem_df['highlight'] + ' ' + lem_df['content'] + ' ' + lem_df['media_desc']
bag_of_words = [x for x in [row for row in bag_of_records] for x in x.split(' ')]

In [14]:
idf_dict = {word:freq for word, freq in dict(Counter(bag_of_words)).items() if freq > 2}

In [15]:
from json import dump

In [16]:
with open('labeling/idf_dict', 'w') as f:
    dump(idf_dict, f)

In [17]:
lem_df.to_csv('labeling/lemmatized_articles.csv', index = False)